In [ ]:
import machine
from random import random
import time

def set_reaction_time(pin):
    global ev_btn
    ev_btn = True

def handler_timer(pin):
    global ev_tim
    ev_tim = True

states = {'IDLE','READY','INVALID PRESS', 'PLAYER RESPONSE' 'VALID PRESS', 'LATE PRESS'}

# Initialize Microcontroller
green_led = machine.Pin('C6', machine.Pin.OUT)
red_led = machine.Pin('C4', machine.Pin.OUT)
s0_button = machine.Pin('B0', machine.Pin.IN, machine.Pin.PULL_UP)
s0_button.irq(trigger = s0_button.IRQ_FALLING, handler = set_reaction_time)
timer = machine.Timer()

#Initialize State Machine
state_ = 'IDLE'
ev_btn = False #button event
ev_tim = False #random timer expire (tr)
t1 = 0
while True:
    if state_ == 'IDLE':
        green_led.value(1)
        red_led.value(1)        
        if ev_btn:
            #print ('pressed')
            state_ = 'READY'
            timer.init(period = int(random()*5000+1000), mode = machine.Timer.ONE_SHOT, callback= handler_timer)
            ev_btn = False             
    if state_ == 'READY':
        green_led.value(0)
        if ev_tim:
            state_ = 'PLAYER RESPONSE'
            red_led.value(0)
            green_led.value(1)
            t1 = time.ticks_ms()
            ev_tim = False
        if ev_btn:
            state_ = 'INVALID PRESS'   
    if state_ == 'PLAYER RESPONSE':
        if ev_btn:
            state_ = 'VALID PRESS'
            ev_btn = False
        if ev_tim:
            state_ = 'LATE PRESS'         
            ev_tim = False
    if state_ == 'VALID PRESS':
        print("Reaction Time:", time.ticks_ms()- t1)
        state_ = 'IDLE'
    if state_ == 'LATE PRESS':
        print("Wake up")
        state_ = 'IDLE'
    if state_ == 'INVALID PRESS':
        print("Trigger happy :-)")
        state_ = 'IDLE'